<a href="https://colab.research.google.com/github/naguzmans/opportunistic-atm/blob/main/pso_model_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2> 2D Particle Swarm Optimization </h2>

In [74]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy import interpolate
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

class Obstacle:
  def __init__(self, x, y, z, r):
    self.x = x
    self.y = y
    self.z = z
    self.r = r

class Particle:
  def __init__(self, origin, destination, handles):
    self.origin = origin
    self.destination = destination

    # Define x array
    self.x, self.y, self.z = self.CreatePath(self.origin, self.destination, handles)

    # Define velocity
    self.vx = np.zeros(handles+2)
    self.vy = np.zeros(handles+2)
    self.vz = np.zeros(handles+2)

    # Calculate Cost
    self.Cost()
    
    # Define Best
    self.Best(self.x, self.y, self.z, self.xs, self.ys, self.zs, self.cost)

  def Cost(self):

    # Create spline
    ts = np.linspace(self.origin[0], self.destination[0], handles+2 )
    spline_x = interpolate.CubicSpline(ts, self.x)
    spline_y = interpolate.CubicSpline(ts, self.y)
    spline_z = interpolate.CubicSpline(ts, self.z)

    ts2 = np.linspace(self.origin[0], self.destination[0], 100)
    self.xs = spline_x(ts2)
    self.ys = spline_y(ts2)
    self.zs = spline_z(ts2)

    dsx = spline_x.derivative(nu=1)
    dsy = spline_y.derivative(nu=1)
    dsz = spline_z.derivative(nu=1)

    self.dxs = dsx(ts2)
    self.dys = dsy(ts2)
    self.dzs = dsz(ts2)
    self.L = np.sum(np.sqrt(self.dxs**2 + self.dys**2 + self.dzs**2))
    
    violation = 0
    for i in range(0, len(obstacles)):
      distance_to_obstacle = np.sqrt((self.xs-obstacles[i].x)**2+(self.ys-obstacles[i].y)**2+(self.zs-obstacles[i].z)**2)
      v = 1-distance_to_obstacle/obstacles[i].r
      for j in range(0,len(v)):
        if v[j] < 0:
          v[j] = 0
      violation += np.mean(v)

    # Calculate final cost
    beta = 100
    # self.cost = np.sum(distance_to_obstacle)*(1+beta*violation)
    self.cost = self.L*(1+beta*violation)

  def Best(self, x, y, z, xs, ys, zs, cost):
    self.best_x = x
    self.best_y = y
    self.best_z = z

    self.best_xs = xs
    self.best_ys = ys
    self.best_zs = zs

    self.best_cost = cost

  def CreatePath(self, start, end, handles):
    x = np.random.uniform(start[0], end[0], handles+2)
    x[0] = start[0]
    x[-1] = end[0]

    y = np.random.uniform(start[1], end[1], handles+2)
    y[0] = start[1]
    y[-1] = end[1]

    z = np.random.uniform(start[1], end[1], handles+2)
    z[0] = start[2]
    z[-1] = end[2]

    return x,y,z

class GlobalBest:
  def __init__(self, x, y, z, xs, ys, zs, cost):
    self.x = x
    self.y = y
    self.z = z
    self.xs = xs
    self.ys = ys
    self.zs = zs
    self.cost = cost
    
# Create model obstacles
obstacles = [Obstacle(1.5,1.5,0,1.5), \
             Obstacle(2,.5,1,.5), \
             Obstacle(.5,2,1,.5), \
             
             #Stack
             Obstacle(3.5,3.5,0,1), \
             Obstacle(3.5,3.5,.5,1), \
             Obstacle(3.5,3.5,1,1),\
             Obstacle(3.5,3.5,1.5,1),\
             Obstacle(3.5,3.5,2,1),\
             Obstacle(3.5,3.5,1.5,1)]

def ms(x, y, z, radius, resolution=10):
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)

data = []
for i in range(0, len(obstacles)):
    (x_pns_surface, y_pns_surface, z_pns_surface) = ms(obstacles[i].x, obstacles[i].y, obstacles[i].z, obstacles[i].r)
    data.append(go.Surface(x=x_pns_surface, y=y_pns_surface, z=z_pns_surface, showscale=False, opacity=1))

# Create Particle Swarm
particles = []
number_of_particles = 50
handles = 3
for i in range(number_of_particles):
  particles.append(Particle((0,0,.5), (5,5,.5), handles))

# Initialize
global_best = GlobalBest(0,0,0,0,0,0,1e6)
for particle in particles:
  if particle.cost < global_best.cost:
    global_best.x = particle.x.copy()
    global_best.y = particle.y.copy()
    global_best.z = particle.z.copy()

    global_best.xs = particle.xs.copy()
    global_best.ys = particle.ys.copy()
    global_best.zs = particle.zs.copy()

    global_best.cost = particle.cost.copy()
    # print(f'Initial Global Best Cost: {global_best.cost}')

# Particle Swarm Optimization
canvas_max_x = 5
canvas_min_x = 0

canvas_max_y = 5
canvas_min_y = 0

canvas_max_z = 5
canvas_min_z = 0

iterations = 50
w = 1 
wdamp = .98
c1 = 1.5
c2 = 1.5
alpha = .1
vel_max_x = alpha*(canvas_max_x - canvas_min_x)
vel_min_x = -vel_max_x
vel_max_y = alpha*(canvas_max_y - canvas_min_y)
vel_min_y = -vel_max_y
vel_max_z = alpha*(canvas_max_z - canvas_min_z)
vel_min_z = -vel_max_z

# plt.figure()
# fig, ax = plt.subplots(figsize=(10,10))

# ax.set_xlabel('Distance (Unit)')
# ax.set_ylabel('Distance (Unit)')
# ax.set_title(f'3D Particle Swarm Optimization \n Trajectory Planning', fontweight='bold')

# ax.set_xlim(canvas_min_x-1,canvas_max_x+1)
# ax.set_ylim(canvas_min_y-1,canvas_max_y+1)
# ax.text(canvas_min_x -.4, canvas_min_y -.4, 'A', fontsize=16, fontweight = 'bold')
# ax.text(canvas_max_x + .2, canvas_max_y + .2, 'B', fontsize=16, fontweight = 'bold')

# for obstacle in obstacles:
#   ax.add_patch(plt.Circle((obstacle.x, obstacle.y), obstacle.r, color='r'))

for i in tqdm(range(0, iterations)):
  
  for particle in particles:

    # X
    particle.vx = w*particle.vx + \
                  c1*np.random.rand(1, handles+2).reshape(handles+2)*(particle.best_x-particle.x) + \
                  c2*np.random.rand(1, handles+2).reshape(handles+2)*(global_best.x-particle.x)
    
    particle.vx = np.maximum(particle.vx.copy(), vel_min_x)
    particle.vx = np.minimum(particle.vx.copy(), vel_max_x)
 
    # particle.x += particle.vx
    particle.x = np.add(particle.x, particle.vx, out=particle.x, casting='unsafe')

    # Y
    particle.vy = w*particle.vy + \
                  c1*np.random.rand(1, handles+2).reshape(handles+2)*(particle.best_y-particle.y) + \
                  c2*np.random.rand(1, handles+2).reshape(handles+2)*(global_best.y-particle.y)
    
    particle.vy = np.maximum(particle.vy.copy(), vel_min_y)
    particle.vy = np.minimum(particle.vy.copy(), vel_max_y)

    # particle.y += particle.vy
    particle.y = np.add(particle.y, particle.vy, out=particle.y, casting='unsafe')

    # Z
    particle.vz = w*particle.vz + \
                  c1*np.random.rand(1, handles+2).reshape(handles+2)*(particle.best_z-particle.z) + \
                  c2*np.random.rand(1, handles+2).reshape(handles+2)*(global_best.z-particle.z)
    
    particle.vz = np.maximum(particle.vz.copy(), vel_min_z)
    particle.vz = np.minimum(particle.vz.copy(), vel_max_z)

    # particle.y += particle.vy
    particle.z = np.add(particle.z, particle.vz, out=particle.z, casting='unsafe')

    # print(f'Iteration {i} - Particle VX: {particle.vx} - Particle VY: {particle.vy}')

    # Calculate cost
    particle.Cost()
    if particle.cost < particle.best_cost:
      particle.best_x = particle.x.copy()
      particle.best_y = particle.y.copy()
      particle.best_z = particle.z.copy()

      particle.best_xs = particle.xs.copy()
      particle.best_ys = particle.ys.copy()
      particle.best_zs = particle.zs.copy()

      particle.best_cost = particle.cost.copy()

      # print(f'Particle Best X: {particle.best_x}')

    if particle.cost < global_best.cost:
      global_best.x = particle.x.copy()
      global_best.y = particle.y.copy()
      global_best.z = particle.z.copy()

      global_best.xs = particle.xs.copy()
      global_best.ys = particle.ys.copy()
      global_best.zs = particle.zs.copy()

      global_best.cost = particle.cost.copy()

      # ax.plot(global_best.xs, global_best.ys, color ='b', alpha=.05)
      # ax.scatter(global_best.x, global_best.y, c = 'b', alpha=.1)

      # print(f'Global Best Cost: {global_best.cost}')

  # print(f'Global Best X Iter: {global_best.x}')
  w = w * wdamp


layout = go.Layout(scene=dict(
                              aspectmode='cube',
                              bgcolor='rgba(0,0,0,0)',
                              xaxis = dict(range=[-0,5], showgrid=True, gridcolor='rgba(0, 0, 0, .1)', backgroundcolor="rgba(0, 0, 0, 0)", showticklabels=True),
                              yaxis = dict(range=[-0,5], showgrid=True, gridcolor='rgba(0, 0, 0, .1)', backgroundcolor="rgba(0, 0, 0, 0)", showticklabels=True),
                              zaxis = dict(range=[-0,5], showgrid=True, gridcolor='rgba(0, 0, 0, .1)', backgroundcolor="rgba(0, 0, 0, 0)", showticklabels=True))
                              )

data.append(go.Scatter3d(x=global_best.xs, y=global_best.ys, z=global_best.zs, mode='lines', marker=dict(size=10, color='red')))
fig = go.Figure(data=data, layout=layout)
fig.update_xaxes(visible=False)

fig.show()

# ax.plot(global_best.xs, global_best.ys, color ='k')
# ax.scatter(global_best.x, global_best.y, c = 'k')
print(f'Final Global Best Cost: {global_best.cost}')


100%|██████████| 50/50 [00:05<00:00,  9.60it/s]


Final Global Best Cost: 155.67796711340048
